In [3]:
# Celda 1: imports y carga
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Cargar datos
df = pd.read_csv(r"C:\Users\melga\vs_private\proyect_sprint7\vehicles_us.csv")

# Vista rápida
print("Filas, columnas:", df.shape)
df.head()


Filas, columnas: (51525, 13)


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [4]:
# inspección rápida de faltantes (resumen)
missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(2)
pd.DataFrame({'missing_count': missing, 'missing_%': missing_pct})


,missing_count,missing_%
is_4wd,25953,50.37
paint_color,9267,17.99
odometer,7892,15.32
cylinders,5260,10.21
model_year,3619,7.02
condition,0,0.00
model,0,0.00
price,0,0.00
fuel,0,0.00
type,0,0.00


In [5]:
# Celda 3: Separar brand y model_name (limpieza básica de text)
# Asumimos: primera palabra = brand, resto = model_name
split = df['model'].str.split(' ', n=1, expand=True)
df['brand'] = split[0].str.capitalize().str.strip()
df['model_name'] = split[1].str.lower().str.strip().fillna('')
# Normalizar la columna 'model' (solo primera palabra)
df['model'] = df['model'].str.split().str[0].str.capitalize()
# Revisión
df[['model', 'brand', 'model_name']].head(10)


,model,brand,model_name
0,Bmw,Bmw,x5
1,Ford,Ford,f-150
2,Hyundai,Hyundai,sonata
3,Ford,Ford,f-150
4,Chrysler,Chrysler,200
5,Chrysler,Chrysler,300
6,Toyota,Toyota,camry
7,Honda,Honda,pilot
8,Kia,Kia,sorento
9,Honda,Honda,pilot


In [6]:
# Celda 4: Estrategia de imputación y limpieza
# Hacer copias para no modificar original en caso de querer revertir
df_clean = df.copy()

# Quitar las filas con NaN en 'model_year', 'cylinders' y 'odometer'
df_clean.dropna(subset=['model_year', 'cylinders', 'odometer'], inplace=True)

# Convertir a tipo entero (int)
df_clean['model_year'] = df_clean['model_year'].astype(int)
df_clean['cylinders'] = df_clean['cylinders'].astype(int)
df_clean['odometer'] = df_clean['odometer'].astype(int)


# paint_color
df_clean['paint_color'] = df_clean['paint_color'].fillna('unknown')

# is_4wd
df_clean['is_4wd'] = df_clean['is_4wd'].fillna(0).astype(int)


# Comprobar resultados
df_clean[['model_year','cylinders','odometer','paint_color','is_4wd']].isna().sum()


model_year     0
cylinders      0
odometer       0
paint_color    0
is_4wd         0
dtype: int64

In [7]:
# Celda 5: Transformaciones adicionales útiles
CURRENT_YEAR = 2025
df_clean['vehicle_age'] = (CURRENT_YEAR - df_clean['model_year']).clip(lower=0)  # evitar negativos
# proteger division por cero/missing
df_clean['price'] = pd.to_numeric(df_clean['price'], errors='coerce').fillna(0)
df_clean['price_per_mile'] = df_clean.apply(lambda r: r['price']/r['odometer'] if r['odometer']>0 else np.nan, axis=1)
# crear categoría de precio rápido
df_clean['price_category'] = pd.cut(df_clean['price'],
                                   bins=[-1, 5000, 15000, 30000, 1000000],
                                   labels=['bajo','medio','alto','lujo'])
# calidad aproximada: usar 'condition' si existe; arreglar mayúsculas/minúsculas
if 'condition' in df_clean.columns:
    df_clean['condition'] = df_clean['condition'].astype(str).str.lower().str.strip().replace({'nan':'unknown'})
else:
    df_clean['condition'] = 'unknown'
    
df_clean.head()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,brand,model_name,vehicle_age,price_per_mile,price_category
0,9400,2011,Bmw,good,6,gas,145000,automatic,SUV,unknown,1,2018-06-23,19,Bmw,x5,14,0.064828,medio
2,5500,2013,Hyundai,like new,4,gas,110000,automatic,sedan,red,0,2019-02-07,79,Hyundai,sonata,12,0.050000,medio
4,14900,2017,Chrysler,excellent,4,gas,80903,automatic,sedan,black,0,2019-04-02,28,Chrysler,200,8,0.184171,medio
5,14990,2014,Chrysler,excellent,6,gas,57954,automatic,sedan,black,1,2018-06-20,15,Chrysler,300,11,0.258653,medio
6,12990,2015,Toyota,excellent,4,gas,79212,automatic,sedan,white,0,2018-12-27,73,Toyota,camry,10,0.163990,medio


In [8]:
# Celda 6: GRAFICO 1 - Sunburst (Marca -> Modelo) por cantidad de autos
fig_sun = px.sunburst(
    df_clean,
    path=['brand', 'model_name'],
    values=None,              # None => cuenta ocurrencias
    title='Sunburst: Conteo de Vehículos por Marca y Modelo (Top brands)',
    color='brand'
)
fig_sun.update_traces(textinfo='label+percent entry')
fig_sun.show()


In [9]:
# Celda 7: Histograma: Condición vs Año del Modelo

fig_hist = px.histogram(
        df_clean,
        x='model_year',
        color='condition',
        title='Histograma de Condición vs Año del Modelo',
        barmode='stack',
        labels={
            'model_year': 'Año del Modelo',
            'condition': 'Condición',
            'count': 'Cantidad'
        }
    )
fig_hist.show()

In [10]:
# Celda 8:Gráfico de Dispersión: Precio vs Año del Modelo
fig_scatter = go.Figure(
        data=[
            go.scatter(
                x=df_clean['model_year'],
                y=df_clean['price'],
                mode='markers',
                marker=dict(
                    size=6,
                    opacity=0.6,
                    color=df_clean['model_year'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(
                        title='Año del Modelo',  # Título de la escala de color
                        titleside='right'
                    )
                )
            )
        ]
    )
# Configurar diseño del gráfico
fig_scatter.update_layout(
        title='Precio vs. Año del Modelo',
        xaxis_title='Año del Modelo',
        yaxis_title='Precio (USD)',
        plot_bgcolor='rgba(0,0,0,0)',
        title_x=0.5  # Centrar título
    )
# Mostrar gráfico
fig_scatter.show()


TypeError: 'module' object is not callable

In [11]:
# Celda 09: Gráfico de Barras Apiladas: Tipos de Vehículos por Modelo
fig_bar = px.bar(
        df_clean,
        x='model',
        color='type',
        title='Tipos de Vehículos por Modelo',
        labels={
            'model': 'Modelo',
            'type': 'Tipo',
            'count': 'Unidades'
        }
    )

    # Ajustes visuales y orden descendente
fig_bar.update_layout(
        barmode='stack',
        xaxis={'categoryorder': 'total descending'},
        title_x=0.5
    )
fig_bar.show()